In [241]:
from fonctions.gestion_bdd import lire_bdd_perso, get_user
from fonctions.visualisation import transformation_stats_visu, plotline_evol_rune_visu
import plotly.graph_objects as go
import pandas as pd
import datetime
import numpy as np

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [242]:
data = lire_bdd_perso('''SELECT sw_user.id, sw_user.joueur, sw_user.visibility, sw_user.guilde_id, sw_user.joueur_id, sw_score.date, sw_score.score, (SELECT guilde from sw_guilde where sw_guilde.guilde_id = sw_user.guilde_id) as guilde
                        FROM sw_user
                        INNER JOIN sw_score ON sw_user.id = sw_score.id
                        where sw_user.visibility != 0''').transpose().reset_index()

    # on transpose la date au format date
data['date'] = pd.to_datetime(data['date'], format="%d/%m/%Y")

In [243]:
data['semaine'] = data['date'].apply(lambda x : datetime.datetime.date(x).isocalendar().week)

In [244]:
data_ranking = data.groupby(['joueur', 'semaine']).agg(
        {'score': 'max'}).reset_index()
data_ranking

,joueur,semaine,score
0,Actarus,40,659
1,Alakar,46,367
2,AllStarChamp,40,1193
3,AllStarChamp,41,1201
4,AllStarChamp,42,1207
...,...,...,...
122,~saru~,44,1274
123,~saru~,46,1302
124,~saru~,48,1314
125,Æthan,42,943


In [245]:
def graph(dataset):
    # dataset['date'] = dataset['date'].apply(lambda x : datetime.date(x).isocalendar().week)
    
    dataset = dataset.groupby(['joueur', 'semaine']).agg(
        {'score': 'max'}).reset_index()

    years = dataset['semaine'].sort_values().unique().tolist()

    
    min_week = min(years)
    max_week = max(years)
    
    min_points = min(dataset['score'])
    max_points = max(dataset['score'])

    # make list of continents
    continents = []
    for continent in dataset["joueur"]:
        if continent not in continents:
            continents.append(continent)
    # make figure
    fig_dict = {
        "data": [],
        "layout": {},
        "frames": []
    }

    # fill in most of layout
    fig_dict["layout"]["xaxis"] = {"range": [min_week-1, max_week+1], "title": 'date'}
    fig_dict["layout"]["yaxis"] = {"title": "GDP per Capita", "type": "log"}
    fig_dict["layout"]["hovermode"] = "closest"
    fig_dict["layout"]["updatemenus"] = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 500, "redraw": False},
                                    "fromcurrent": True, "transition": {"duration": 300,
                                                                        "easing": "quadratic-in-out"}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]

    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Year:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }

    # make data
    year = years[0]
    for continent in continents:
        dataset_by_year = dataset[dataset['semaine'] == year]
        dataset_by_year_and_cont = dataset_by_year[
            dataset_by_year["joueur"] == continent]

        data_dict = {
            "x": list(dataset_by_year_and_cont['semaine']),
            "y": list(dataset_by_year_and_cont["score"]),
            "mode": "markers",
            "text": list(dataset_by_year_and_cont["joueur"]),
            "name": continent
        }
        fig_dict["data"].append(data_dict)

    # make frames
    for year in years:

        frame = {"data": [], "name": str(year)}
        for continent in continents:
            dataset_by_year = dataset[dataset['semaine'] == year]
            dataset_by_year_and_cont = dataset_by_year[
                dataset_by_year["joueur"] == continent]

            data_dict = {
                "x": list(dataset_by_year_and_cont['semaine']),
                "y": list(dataset_by_year_and_cont["score"]),
                "mode": "markers",
                "text": list(dataset_by_year_and_cont["joueur"]),
                "name": continent
            }
            frame["data"].append(data_dict)

        fig_dict["frames"].append(frame)
        slider_step = {"args": [
            [str(year)],
            {"frame": {"duration": 300, "redraw": False},
            "mode": "immediate",
            "transition": {"duration": 300}}
        ],
            "label": str(year),
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)


    fig_dict["layout"]["sliders"] = [sliders_dict]

    fig = go.Figure(fig_dict)

    fig.show()

In [246]:
graph(data)